In [1]:
from PPO_mountaincar import *
import gym
import pickle


In [2]:
env = gym.make("MountainCar-v0")
agent1 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent1.load_models("model/best_actor_model_MC")
agent2 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent2.load_models("model/half_actor_model_MC")


... loading models ...
... loading models ...


In [17]:
K = 30
 
trajectory_record = [] # [(t_1,t_2)_1,...,(t_1,t_2)_K]
reward_record = []  # [(R_1,R_2)_1,...,(R_1,R_2)_K]
trajectory1_record = []
trajectory2_record = []

def softmax(x,y):
    return np.exp(x)/(np.exp(x)+np.exp(y))

for k in range(K):
    cum_reward1 = 0
    cum_reward2 = 0
    trajectory_buffer1 = []  
    trajectory_buffer2 = [] 
    
    # agent1
    state, _ = env.reset()
    done = False
    
    while not done:
        action, probs, value = agent1.choose_action(state)
        trajectory_buffer1.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward1 += reward
        done = terminated

    # agent2
    state, _ = env.reset()
    done = False

    while not done:
        action, probs, value = agent2.choose_action(state)
        trajectory_buffer2.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward2 += reward
        done = terminated

    trajectory_record.append([trajectory_buffer1,trajectory_buffer2])
    trajectory1_record.append(trajectory_buffer1)
    trajectory2_record.append(trajectory_buffer2)
    reward_record.append([cum_reward1, cum_reward2])

print(reward_record)
probs = [[softmax(x[0],x[1]),softmax(x[1],x[0])] for x in reward_record]
label = []
for prob in probs:
    label.append(0 if prob[0]>prob[1] else 1)

save_data = []
for traj, label in zip(trajectory_record,label):
    save_data.append((traj[0],traj[1], label))


[[-144.0, -7180.0], [-143.0, -4389.0], [-171.0, -1347.0], [-177.0, -3874.0], [-148.0, -3585.0], [-178.0, -4323.0], [-152.0, -3812.0], [-143.0, -11983.0], [-223.0, -3222.0], [-263.0, -2469.0], [-137.0, -7119.0], [-211.0, -2996.0], [-187.0, -3670.0], [-163.0, -2554.0], [-146.0, -2324.0], [-277.0, -1652.0], [-142.0, -1057.0], [-246.0, -3824.0], [-139.0, -2860.0], [-178.0, -1520.0], [-225.0, -7056.0], [-210.0, -4567.0], [-212.0, -2668.0], [-200.0, -5113.0], [-140.0, -1852.0], [-205.0, -2511.0], [-210.0, -5698.0], [-184.0, -3102.0], [-206.0, -3785.0], [-256.0, -1586.0]]


## zahra's framework

### save traj

In [18]:
with open("traj_record/trajectory_record.pkl", "wb") as f:
    pickle.dump(save_data, f)

### load traj

In [ ]:
with open("traj_record/trajectory_record.pkl", "rb") as f:
    data = pickle.load(f)

_data = data

## my suggestion

### save traj

In [20]:
with open("traj_record/trajectory_record_yugo.pkl", "wb") as f:
    pickle.dump({"trajectory1": trajectory1_record,"trajectory2": trajectory2_record, "label": label}, f)

### load traj

In [22]:
with open("traj_record/trajectory_record_yugo.pkl", "rb") as f:
    data = pickle.load(f)

# Access contents
_trajectory1_record = data["trajectory1"]
_trajectory2_record = data["trajectory2"]
_label = data["label"]